In [1]:
###### Comment Legend ######

    # Describing a function
    ### Describing an issue
    ###### Infomational ######

###### To Do List ######

    # Create a function to save bad images for later use

In [1]:
import cv2
import os
import math
import mmcv
import torch
import pandas as pd
from time import sleep
from datetime import datetime
import numpy as np
from matplotlib import pyplot as plt
from ultralytics import YOLO
from ultralytics.yolo.utils.plotting import Annotator

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

%run utils/superimposition/functions.ipynb
# %run utils/extraction/objClass.ipynb

C:\Users\gmald\anaconda3\envs\senior\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
class Configuration:
    def __init__(self, minimal_img, main_img, seg_net, video_format, fps, file_name, skip_frames):
        self.minimal_img = minimal_img
        self.main_img = main_img
        self.seg_net = seg_net
        self.fourcc = cv2.VideoWriter_fourcc(*video_format)
        self.fps = fps
        self.main_path = f"videos/{file_name}/"
        self.processed_path = self.main_path + "processed/"
        self.skip_frames = skip_frames

# List available video files
file_list = os.listdir("videos")
selected_file = file_list[3]

print(file_list, "\nSelected File:", selected_file)

# Create a configuration object
config = Configuration(minimal_img="Thermal", main_img="RGB", seg_net="utils/segmentation/yolov8n-seg.pt", video_format="mp4v", fps=10, file_name=selected_file, skip_frames=80)

['2022-12-01_18-03-33', '2022-12-02_13-05-04', '2023-01-24_15-28-00', '2023-01-28_15-50-17', '2023-04-18_16-00-58', '2023-04-18_16-08-43', '2023-04-18_16-24-54', '2023-04-27_12-30-56', '2023-04-27_12-35-33', 'combined_outisde_13.gif', 'newStereo', 'outside_test', 'SII'] 
Selected File: 2023-01-28_15-50-17


In [3]:
# Video Processor

# Load the segmentation model
seg_model = YOLO(config.seg_net)

# Open video files
rgb_video = cv2.VideoCapture(f"{config.main_path}rgbout.mp4")
thermal_video = cv2.VideoCapture(f"{config.main_path}thermalout.mp4")
depth_video = cv2.VideoCapture(f"{config.main_path}stereoout.mp4")
left_video = cv2.VideoCapture(f"{config.main_path}leftout.mp4")

# Initialize frame counters
calibration_count = 0
frame_count = 0

# Set the initial frame position for each video
rgb_video.set(cv2.CAP_PROP_POS_FRAMES, config.skip_frames)
thermal_video.set(cv2.CAP_PROP_POS_FRAMES, config.skip_frames)
depth_video.set(cv2.CAP_PROP_POS_FRAMES, config.skip_frames)
left_video.set(cv2.CAP_PROP_POS_FRAMES, config.skip_frames)

ten_scales = [[1, 1, 1]]
ten_offsets = [[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]]

images = {"RGB": None, "Thermal": None, "Left": None, "Depth": None}

# Main loop to process video frames
while True:
    # Read a frame from each video
    ret, images["RGB"] = rgb_video.read()
    ret1, images["Thermal"] = thermal_video.read()
    ret2, images["Depth"] = depth_video.read()
    ret3, images["Left"] = left_video.read()
    if not (ret & ret1 & ret2 & ret3):
        break
    
    depth_sp = images["Depth"].shape
    # Stretch depth to deal with distortion  
    images["Depth"] = cv2.resize(images["Depth"], (int(depth_sp[1]*1.2), depth_sp[0]))

    # Perform calibration every 2 frames
    if calibration_count % 20 == 0:
        scales, offset = calibrate_images(images.copy(), seg_model, config.minimal_img, depth_bias=0.85) # 0.85)

        if scales is not None and offset is not None:
            if ten_scales[0] == [1, 1, 1]:
                del ten_scales[0]
                del ten_offsets[0]

            ten_scales.append(scales)
            ten_offsets.append(offset)

        if len(ten_scales) > 30:
            random_delete = int(np.random.rand() * 30)
            
            del ten_scales[random_delete]
            del ten_offsets[random_delete]

        scales, offset = filter_offsets_scales(ten_scales, ten_offsets)
        scales = {"RGB": scales[0], "Thermal": scales[1], "Depth": scales[2]}
        offset = {"RGB": offset[0], "Thermal": offset[1], "Depth": offset[2]}
        offset["Depth"][2] += 15
        
        # If manual Input is needed add here
        # scales = {'RGB': 0.3300825220683977, 'Thermal': 1.0, 'Depth': 1.0634868466154548}
        # offset = {'RGB': [53, 102, 83, 72], 'Thermal': [0, 0, 0, 0], 'Depth': [16, 72, 50, 51]}

    # Convert stereo frame to grayscale for visualization
    images["Depth"] = cv2.cvtColor(images["Depth"], cv2.COLOR_BGR2GRAY)

    # Modify images based on calibration settings
    images["Unrelated"] = superimpose_images(images.copy(), add_color=True)
    images = modify_images(images.copy(), scales, offset, images[config.minimal_img].shape[0])

    images["Related"] = superimpose_images(images.copy(), add_color=True)

    stereo_frame_mapped = cv2.normalize(images["Depth"], None, 255, 0, cv2.NORM_INF, cv2.CV_8UC1)
    stereo_frame_mapped = cv2.equalizeHist(stereo_frame_mapped)
    stereo_frame_mapped = cv2.applyColorMap(stereo_frame_mapped, cv2.COLORMAP_JET)
    images["Color_Depth"] = stereo_frame_mapped

    # Create video files for processed output
    if calibration_count == 0:
        thermal_out = cv2.VideoWriter(f"{config.processed_path}Thermal_sii.mp4", config.fourcc, config.fps, (images[config.minimal_img].shape[:2][::-1]))
        stereo_out = cv2.VideoWriter(f"{config.processed_path}Depth_sii.mp4", config.fourcc, config.fps, (images[config.minimal_img].shape[:2][::-1]), 0) # For GrayScale
        stereo_color_out = cv2.VideoWriter(f"{config.processed_path}Depth_color_sii.mp4", config.fourcc, config.fps, (images[config.minimal_img].shape[:2][::-1])) # For GrayScale
        rgb_out = cv2.VideoWriter(f"{config.processed_path}RGB_sii.mp4", config.fourcc, config.fps, (images[config.minimal_img].shape[:2][::-1]))
        related_sii_out = cv2.VideoWriter(f"{config.processed_path}Related_SII.mp4", config.fourcc, config.fps, (images[config.minimal_img].shape[:2][::-1]))
        unrelated_sii_out = cv2.VideoWriter(f"{config.processed_path}Unrelated_SII.mp4", config.fourcc, config.fps, (images[config.minimal_img].shape[:2][::-1]))

    # Resize related and unrelated images
    images["Related"] = cv2.resize(images["Related"], images[config.minimal_img].shape[:2][::-1])
    images["Unrelated"] = cv2.resize(images["Unrelated"], images[config.minimal_img].shape[:2][::-1])

    # Display images
    for label in images:
        cv2.imshow(label, images[label])

    # Write frames to output video files
    thermal_out.write(images["Thermal"])
    stereo_out.write(images["Depth"])
    stereo_color_out.write(images["Color_Depth"])
    rgb_out.write(images["RGB"])
    related_sii_out.write(images["Related"])
    unrelated_sii_out.write(images["Unrelated"])

    # Increment frame counters
    calibration_count += 1
    frame_count += 1

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
print("\nProcessing Ended. Saving videos...")
        
# Release video files
rgb_out.release()
thermal_out.release()
stereo_out.release()
stereo_color_out.release()
related_sii_out.release()
unrelated_sii_out.release()

print("Saving complete!")

cv2.destroyAllWindows()


Processing Ended. Saving videos...
Saving complete!


In [ ]:
# If there was some kind of issue you should be able to recover videos
rgb_out.release()
thermal_out.release()
stereo_out.release()
related_sii_out.release()
unrelated_sii_out.release()

In [62]:
print(scales, offset)

{'RGB': 0.3300825220683977, 'Thermal': 1.0, 'Depth': 1.0634868466154548} {'RGB': [53, 102, 83, 72], 'Thermal': [0, 0, 0, 0], 'Depth': [16, 72, 50, 51]}
